# **Model Regresi Harga Perumahan Boston Menggunakan FCNN**

Penelitian ini akan memprediksi harga rata-rata rumah di pinggiran kota Boston tertentu pada pertengahan 1970-an menggunakan metode fully connected neural network (FCNN). Beberapa faktor yang mempengaruhi harga rata-rata rrumah di pinggiran kota pada saat itu adalah tingkat kejahatan, tarif pajak properti lokal, dan lain sebagainya.

## Dataset

Dataset yang akan digunakan memiliki total series 506, dan terbagi menjadi 404 sampel pelatihan dan 102 sampel pengujian. Setiap fitur dalam data input (seperti tingkat kejahatan) memiliki skala yang berbeda-beda. Misalnya proporsi, yang mengambil nilai antara 0 dan 1, ada juga yang mengambil nilai antara 1 dan 12, dan ada juga yang mengambil nilai antara 0 dan 100. Dataset yang akan digunakan memiliki 404 sampel pelatihan dan 102 sampel pengujian. Dataset ini memiliki 13 fitur.

In [ ]:
# Import library Keras terlebih dahulu.
import keras
from keras.datasets import boston_housing
keras.__version__

In [ ]:
# Load dataset yang akan digunakan.
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
# Jumlah data training sebanyak 404 sampel dan memiliki 13 fitur.
train_data[1],train_data.shape

In [ ]:
# Jumlah data testing sebanyak 102 sampel dan memiliki 13 fitur.
test_data.shape

In [ ]:
# Targetnya adalah nilai median rumah yang ditempati pemilik (dalam ribuan dolar).
# Harga berkisar antara $ 10.000 hingga $ 50.000.
import pandas as pd
df_train_targets = pd.DataFrame(train_targets)
train_targets

In [ ]:
# Awalnya peneliti fokus pada variabel terikat, karena sifat variabel terikat sangat penting
# untuk pemilihan model, yaitu variabel harga rumah yang juga jenis variabel kontinyu.
# Untuk memahaminya kita plot terlebih dahulu distribusinya.
# Data train_targets diplot ke histogram terlebih dahulu menggunakan library Seaborn.
import pandas as pd
import seaborn as sns
sns.distplot(df_train_targets, color='red');

In [ ]:
# Kemudian dilihat sebaran variabel bebasnya (fitur) sebanyak 13 variabel.
# Histogram masing-masing fitur dapat dilihat pada grafik di bagian diagonal.
# Data train_data diplot menggunakan library Seaborn.
import pandas as pd
df_train_data = pd.DataFrame(train_data)
sns.pairplot(df_train_data);

In [ ]:
# Kemudian dilakukan normalisasi fitur. Untuk setiap fitur dalam data input,
# peneliti mengurangi rata-rata fitur dan membaginya dengan standar deviasi,
# sehingga fitur akan berpusat di sekitar 0 dan memiliki simpangan baku yang standar.
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

## Pemodelan
Karena sampel tergolong kecil, peneliti menggunakan network yang kecil dengan 2 hidden layer, masing-masing sebanyak 128 dense unit (kondisi optimal). Umumnya, semakin sedikit data pelatihan maka akan semakin overfitting. Untuk itu, menggunakan network kecil merupakan salah satu cara untuk mengurangi overfitting.

In [ ]:
# Peneliti menggunakan fungsi dari library Keras untuk membangun model.
# Optimizer yang digunakan adalah RMSprop dari library Keras.
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

model = build_model()

In [ ]:
# Visualisasi struktur model.
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='struktur_model_fcnn.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Ringkasan struktur model.
print(model.summary())

## Pelatihan dan Validasi

Untuk mengevaluasi model jaringannya, kami terus menyesuaikan parameternya (seperti jumlah epoch yang digunakan untuk pelatihan). Data dibagi menjadi satu set pelatihan dan satu set validasi. Namun, karena datanya tergolong sangat sedikit, set validasi akan menjadi sangat kecil (sekitar 100 sampel). Konsekuensinya adalah skor validasi dapat berubah banyak tergantung pada titik data mana yang digunakan untuk validasi dan yang digunakan untuk pelatihan. Skor validasi kemungkinan juga akan memiliki varians tinggi sehubungan dengan hal tersebut.

Dari beberapa sumber, praktik terbaik untuk situasi seperti itu adalah menggunakan validasi silang K-Fold, yaitu membagi data yang tersedia menjadi K partisi (umumnya K=4 atau K=5), kemudian membuat model K yang identik, dan melatih masing-masing pada partisi K-1 sambil mengevaluasi partisi yang tersisa. Nilai validasi untuk model yang digunakan selanjutnya akan menjadi rata-rata dari nilai validasi K yang diperoleh.

In [ ]:
# Peneliti melatih network menggunakan epoch 100 dengan fold 5 (kondisi optimal).
# Untuk mencatat seberapa baik model di setiap epoch, log skor validasi akan
# disimpan untuk setiap epoch.
import numpy as np
from keras import backend as K

# Memory clean-up.
K.clear_session()

k = 5
num_val_samples = len(train_data) // k

num_epoch = 100
num_batch_size = 5
all_mae_histories = []

for i in range(k):
    print('Memproses Fold #', i+1)
    # Menyiapkan data validasi (data dari partisi ke-K).
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    # Menyiapkan data pelatihan (data dari semua partisi lain).
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    # Membangun model dengan fungsi dari library Keras.
    model = build_model()
    # Melatih model (dalam silent mode, verbose=0).
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epoch, batch_size=num_batch_size, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

In [ ]:
# Menghitung rata-rata skor MAE per epoch untuk semua fold.
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epoch)]
    
#average_mae_history
np.mean(average_mae_history)

In [ ]:
# Kemudian hasilnya diplot.
import matplotlib.pyplot as plt
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epoch')
plt.ylabel('MAE Hasil Validasi')
plt.show()

## Pengujian dan Evaluasi

In [ ]:
# Menurut plot di atas, MAE hasil validasi berhenti menurun secara signifikan setelah 10 epoch.
# Peneliti melakukan pengujian sesuai dengan hyperparameter pada kondisi pelatihan.
K.clear_session()

epoch_test = 100
batch_size_test = 5

model = build_model()
history = model.fit(train_data, train_targets,
          epochs=epoch_test, batch_size=batch_size_test, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
hist = pd.DataFrame(history.history['mae'])

In [ ]:
# Melihat skor MAE hasil pengujian.
test_mae_score

In [ ]:
# Melihat grafik skor MAE data pengujian versus data validasi.
def plot_history():
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.plot(hist, label='Pengujian')
    plt.plot(range(1, len(average_mae_history) + 1), average_mae_history, label='Validasi')
    plt.legend()
    plt.ylim([0.5,6])
    plt.xlim()

plot_history()

In [ ]:
# Melihat plot nilai riil versus nilai prediksi.
test_prediction = model.predict(test_data).flatten()
def plot_a():
    plt.ylabel('Nilai Prediksi')
    plt.xlabel('Nilai Riil')
    plt.scatter(test_targets, test_prediction, color='green')
    plt.plot([0,50], [0,50], color='red')

plot_a()